In [1]:
import os
from urllib.request import urlretrieve
from zipfile import ZipFile
import shutil

github_url = "https://github.com/Mateorovere/TP2_NLP/archive/main.zip"
zip_file_path, _ = urlretrieve(github_url, "TP2_NLP.zip")

with ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall("/content")

os.remove(zip_file_path)

carpeta_origen = "/content/TP2_NLP-main/Libros"

carpeta_destino = 'llamaindex_data'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

shutil.rmtree('/content/TP2_NLP-main')


In [2]:
!pip install llama_index sentence-transformers pypdf langchain python-decouple llama-index chromadb

In [35]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from jinja2 import Template
import requests
from decouple import config
import chromadb


def zephyr_instruct_template(messages, add_generation_prompt=True):
    # Definir la plantilla Jinja
    template_str = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    template = Template(template_str)

    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)

def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    try:
        api_key = 'tu_api_de_hugging_face'

        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
        # Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        response = requests.post(api_url, headers=headers, json=data)

        respuesta = response.json()[0]["generated_text"][len(prompt):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

def prepare_prompt(query_str: str, nodes: list):
  TEXT_QA_PROMPT_TMPL = (
      "La información de contexto es la siguiente:\n"
      "---------------------\n"
      "{context_str}\n"
      "---------------------\n"
      "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n"
      "Pregunta: {query_str}\n"
      "Respuesta: "
  )

  # Construimos el contexto de la pregunta
  context_str = ''
  for node in nodes:
      page_label = node.metadata["page_label"]
      file_path = node.metadata["file_path"]
      context_str += f"\npage_label: {page_label}\n"
      context_str += f"file_path: {file_path}\n\n"
      context_str += f"{node.text}\n"

  messages = [
      {
          "role": "system",
          "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
      },
      {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)},
  ]

  final_prompt = zephyr_instruct_template(messages)
  return final_prompt

print('Cargando modelo de embeddings...')
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'))

print('Indexando documentos...')

documents = SimpleDirectoryReader("llamaindex_data").load_data()

#Si bien probe con ChromaDB, los resultados siempre fueron superiores con VectorStoreIndex

# embedds = []
# texts = []
# metadata = []
# id_ = []

# for doc in documents:
#   texts.append(doc.text)
#   metadata.append(doc.metadata)
#   id_.append(doc.id_)
#   embedding = embed_model.get_text_embedding(doc.text)
#   embedds.append(embedding)

# client = chromadb.Client()
# collection = client.get_or_create_collection("anatomicon")

# collection.add(
#   documents=texts,
#   metadatas=metadata,
#   ids=id_,
#   embeddings=embedds
# )


index = VectorStoreIndex.from_documents(documents, show_progress=True,
                                        service_context=ServiceContext.from_defaults(embed_model=embed_model, llm=None))


retriever = index.as_retriever(similarity_top_k=2)

print('Realizando llamada a HuggingFace para generar respuestas...\n')

queries = ['¿Que es la anatomia humana?',
           '¿De que esta compuesto un hueso?',
           '¿Que funciones tiene un vaso sanguineo?']

for query_str in queries:
    # Traemos los documentos más relevantes para la consulta
    nodes = retriever.retrieve(query_str)
    final_prompt = prepare_prompt(query_str, nodes)
    print('Pregunta:', query_str)
    print('Respuesta:')
    print(generate_answer(final_prompt))
    print('-------------------------------------------------------')

Cargando modelo de embeddings...
Indexando documentos...
LLM is explicitly disabled. Using MockLLM.


Parsing nodes:   0%|          | 0/347 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/597 [00:00<?, ?it/s]

Realizando llamada a HuggingFace para generar respuestas...

Pregunta: ¿Que es la anatomia humana?
Respuesta:
La anatomía humana se refiere al estudio del cuerpo humano en términos de su estructura física y funcional. Se puede dividir en varias disciplinas específicas, como la anatomía sistémica normal, que estudia los sistemas del cuerpo, la anatomía descriptiva topográfica, que considera el cuerpo dividido en regiones, la anatomía funcional, que se centra en la relación entre las formas y funciones del cuerpo, la anatomía de superficie, que estudia la configuración externa del cuerpo, la anatomía comparada, que compara las disposiciones anatómicas y estructurales de ciertos órganos o sistemas del cuerpo humano con los de otras especies de animales, la anatomía imagenológica, que interpreta las imágenes radiográficas, de resonancia magnética nuclear y de ecografías, entre otras técnicas, la anatomía endoscópica, que estudia partes internas del cuerpo a través de artefactos introducido